In [1]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [10]:
df = pd.read_csv("CB-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 1009

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,...,defint,defyds,defavg,deftds,defpd,hasbling,heisman,allamerican,hasawards,drafted
0,1997,antonio banks,Virginia Tech,CB,70.0,203,4.66,18.0,36.0,117.0,...,8.0,150.0,18.8,1.0,NaN,0.0,0.0,0.0,0.0,Y
1,1997,ronde barber,Virginia,CB,69.4,185,4.68,14.0,34.5,118.0,...,15.0,112.0,7.5,0.0,NaN,0.0,0.0,0.0,0.0,Y
2,1997,michael booker,Nebraska,CB,73.3,203,NaN,10.0,NaN,NaN,...,4.0,58.0,14.5,0.0,NaN,0.0,0.0,0.0,0.0,Y
3,1997,byron capers,Florida State,CB,73.0,202,4.66,14.0,31.0,116.0,...,2.0,10.0,5.0,0.0,NaN,0.0,0.0,0.0,0.0,Y
4,1997,kevin devine,California,CB,69.4,184,4.53,20.0,NaN,NaN,...,5.0,19.0,3.8,0.0,NaN,0.0,0.0,0.0,0.0,N


In [7]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [11]:
for index, row in df.iterrows():
    if (df.loc[index,"hasbling"] != 1.0):
        if (df.loc[index,"hasbling"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                try:
                    df.loc[index, "solotackles"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='tackles_solo']").text
                except:
                    df.loc[index, "solotackles"] = np.nan

                try:
                    df.loc[index, "assisttackles"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='tackles_assist']").text
                except:
                    df.loc[index, "assisttackles"] = np.nan

                try:
                    df.loc[index, "totaltackles"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='tackles_total']").text
                except:
                    df.loc[index, "totaltackles"] = np.nan

                try:
                    df.loc[index, "defint"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int']").text
                except:
                    df.loc[index, "defint"] = np.nan

                try:
                    df.loc[index, "defyds"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_yds']").text
                except:
                    df.loc[index, "defyds"] = np.nan

                try:
                    df.loc[index, "defavg"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_yds_per_int']").text
                except:
                    df.loc[index, "defavg"] = np.nan

                try:
                    df.loc[index, "deftds"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_td']").text
                except:
                    df.loc[index, "deftds"] = np.nan

                try:
                    df.loc[index, "defpd"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='pass_defended']").text

                except:
                    df.loc[index, "defpd"] = np.nan
                    
                if (len(browser.find_by_css("ul#bling")) > 0):
                    df.loc[index,"hasbling"] = 1
                else:
                    df.loc[index,"hasbling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    df.loc[index,"allamerican"] = 1
                else:
                    df.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    df.loc[index,"heisman"] = 1
                else:
                    df.loc[index,"heisman"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"hasawards"] = 1
                else:
                    df.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"hasbling"] = 0
                df.loc[index,"allamerican"] = 0
                df.loc[index,"heisman"] = 0
                df.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            try:
                                df.loc[index, "solotackles"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='tackles_solo']").text
                            except:
                                df.loc[index, "solotackles"] = np.nan
                            
                            try:
                                df.loc[index, "assisttackles"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='tackles_assist']").text
                            except:
                                df.loc[index, "assisttackles"] = np.nan
                            
                            try:
                                df.loc[index, "totaltackles"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='tackles_total']").text
                            except:
                                df.loc[index, "totaltackles"] = np.nan
                            
                            try:
                                df.loc[index, "defint"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int']").text
                            except:
                                df.loc[index, "defint"] = np.nan
                                
                            try:
                                df.loc[index, "defyds"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_yds']").text
                            except:
                                df.loc[index, "defyds"] = np.nan
                            
                            try:
                                df.loc[index, "defavg"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_yds_per_int']").text
                            except:
                                df.loc[index, "defavg"] = np.nan
                            
                            try:
                                df.loc[index, "deftds"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='def_int_td']").text
                            except:
                                df.loc[index, "deftds"] = np.nan
                            
                            try:
                                df.loc[index, "defpd"] = browser.find_by_css("table#defense tfoot tr.bold")[0].find_by_css("td[data-stat='pass_defended']").text

                            except:
                                df.loc[index, "defpd"] = np.nan
                            
                            if (len(browser.find_by_css("ul#bling")) > 0):
                                df.loc[index,"hasbling"] = 1
                            else:
                                df.loc[index,"hasbling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                df.loc[index,"allamerican"] = 1
                            else:
                                df.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                df.loc[index,"heisman"] = 1
                            else:
                                df.loc[index,"heisman"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"hasawards"] = 1
                            else:
                                df.loc[index,"hasawards"] = 0
                            break
                        else:
                            df.loc[index,"hasbling"] = 0
                            df.loc[index,"allamerican"] = 0
                            df.loc[index,"heisman"] = 0
                            df.loc[index,"hasawards"] = 0
                            break
                    else:
                        df.loc[index,"hasbling"] = 0
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"hasawards"] = 0

In [12]:
df.to_csv("CB-Train.csv", index=False)